---

<div style="text-align:center;">

# **[EVENT-BASED SNN for DAVISDATA](https://tonic.readthedocs.io/en/latest/generated/tonic.datasets.DAVISDATA.html)**

</div>

---


### Citations


@article{eshraghian2021training,
title = {Training spiking neural networks using lessons from deep learning},
author = {Eshraghian, Jason K and Ward, Max and Neftci, Emre and Wang, Xinxin
and Lenz, Gregor and Dwivedi, Girish and Bennamoun, Mohammed and
Jeong, Doo Seok and Lu, Wei D},
journal = {Proceedings of the IEEE},
volume = {111},
number = {9},
pages = {1016--1054},
year = {2023}
}

@article{mueggler2017event,
title = {The event-camera dataset and simulator: Event-based data for pose estimation, visual odometry, and SLAM},
author = {Mueggler, Elias and Rebecq, Henri and Gallego, Guillermo and Delbruck, Tobi and
Scaramuzza, Davide},
journal = {The International Journal of Robotics Research},
volume = {36},
number = {2},
pages = {142--149},
year = {2017},
publisher = {SAGE Publications Sage UK: London, England}
}


### Importing Necessary Libraries


In [5]:
! pip install snntorch --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 1.8 MB/s eta 0:00:00


In [6]:
import tonic

dataset = tonic.datasets.DAVISDATA(save_to="/data", recording="all")

data, targets = dataset[0]
events, imu, images = data

  0%|          | 0/1824397508 [00:00<?, ?it/s]

  0%|          | 0/2513063396 [00:00<?, ?it/s]

  0%|          | 0/1553619361 [00:00<?, ?it/s]

  0%|          | 0/368439195 [00:00<?, ?it/s]

  0%|          | 0/829698602 [00:00<?, ?it/s]

...

  0%|          | 0/18154269 [00:00<?, ?it/s]

  0%|          | 0/51370469 [00:00<?, ?it/s]

  0%|          | 0/50537743 [00:00<?, ?it/s]

  0%|          | 0/37984795 [00:00<?, ?it/s]

  0%|          | 0/82696872 [00:00<?, ?it/s]

In [7]:
import numpy as np

mean_diff = np.diff(list(zip(images["ts"], images["ts"][1:]))).mean()
print(f"Average difference in image timestamps in microseconds: {mean_diff}")

Average difference in image timestamps in microseconds: 46065.511179645335


In [8]:
import torch
import torchvision

sensor_size = tonic.datasets.DAVISDATA.sensor_size
frame_transform = tonic.transforms.ToFrame(
    sensor_size=sensor_size, time_window=mean_diff
)

image_center_crop = torchvision.transforms.Compose(
    [torch.tensor, torchvision.transforms.CenterCrop((100, 100))]
)

# implement some padding here before data_transform to slice recordings to all the same size?
def data_transform(data):
    # first we have to unpack our data
    events, imu, images = data
    # we bin events to event frames
    frames = frame_transform(events)
    # then we can apply frame transforms to both event frames and images at the same time
    frames_cropped = image_center_crop(frames)
    images_cropped = image_center_crop(images["frames"])
    return frames_cropped, imu, images_cropped

In [9]:
print("frames_cropped: ", len(frames_cropped))
print("images_cropped: ", len(images_cropped))
print("imu: ", len(imu))

NameError: ignored

In [ ]:
dataset = tonic.datasets.DAVISDATA(
    save_to="/data/transformed", recording="all", transform=data_transform
)

data, targets = dataset[0]
frames_cropped, imu, images_cropped = data

In [ ]:
print("frames_cropped: ", len(frames_cropped))
print("images_cropped: ", len(images_cropped))
print("imu: ", len(imu))

frames_cropped:  1298
images_cropped:  1298
imu:  6


In [14]:
from sklearn.model_selection import train_test_split

# Cached Split - Training: 20% | Validation: 80%
# using cached_dataset as our data
indices = list(range(len(dataset)))
train_indices, valid_indices = train_test_split(
    indices, test_size=0.2, random_state=42
)  # random_state is to ensure random splitting

# creating training and validation subsets
train_dataset = torch.utils.data.Subset(dataset, train_indices)
valid_dataset = torch.utils.data.Subset(
    dataset, valid_indices
)  # use 'train_dataset' and 'valid_dataset' for training and validation

### DataLoader


In [15]:
# Fast Dataloading, trying some stuff out
from torch.utils.data import DataLoader
from tonic import DiskCachedDataset

# was trying to cache,  encountered issues w certain files
# cached_trainset = DiskCachedDataset(train_dataset, cache_path='/data/cache')
# cached_dataloader = DataLoader(cached_trainset)

batch_size = 128
# trainloader = DataLoader(cached_trainset, batch_size=128, collate_fn=tonic.collation.PadTensors()) # not using this then

In [16]:
print(str(targets))
# print(str(trainloader))

{'ts': array([       0,     5000,     9999, ..., 59760502, 59765502, 59770501]), 'point': array([[0.39737836, 1.46349662, 1.27700999],
       [0.39737453, 1.46358351, 1.27689442],
       [0.39741669, 1.46355683, 1.27682535],
       ...,
       [0.80376061, 0.87389838, 0.88812912],
       [0.80031929, 0.87850653, 0.88760131],
       [0.79677809, 0.88302023, 0.88691498]]), 'rotation': array([[ 0.00738978, -0.06762916,  0.88567912, -0.45928661],
       [ 0.00728549, -0.06771989,  0.8857668 , -0.45910578],
       [ 0.00709958, -0.06783558,  0.88575929, -0.45910609],
       ...,
       [-0.12342553, -0.36257901,  0.86528494, -0.32339537],
       [-0.11898631, -0.36138919,  0.86582889, -0.32493145],
       [-0.11484748, -0.35884514,  0.86677933, -0.32670143]]), 'rosbagType': 'geometry_msgs_PoseStamped'}


In [17]:
# DataLoader for training
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, collate_fn=tonic.collation.PadTensors()) #, collate_fn=tonic.collation.PadTensors())

# DataLoader for testing
test_loader = DataLoader(valid_dataset, batch_size=128, shuffle=False)


In [18]:
print(len(valid_dataset))
print(str(train_loader))
# event_tensor, target = next(iter(train_loader))
# print(event_tensor.shape)

5


In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms

# 1xHxW?
input_size = 1
input_height = 100
input_width = 100


# Define a simple CNN architecture (update layers, inputs/outputs to match current data transform approach)
class PoseEstimationCNN(nn.Module):
    def __init__(self):
        super(PoseEstimationCNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=input_size, out_channels=32, kernel_size=3, stride=1, padding=1
        )
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(
            in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1
        )
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(64 * (input_height // 4) * (input_width // 4), 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128,2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = self.flatten(x)

        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)

        return x

In [ ]:
# Instantiate the model
model = PoseEstimationCNN()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# trying to figure out which tuple kind is being accessed
# for sample, target in batch:
#     print("Sample:", sample)
#     print("Target:", target)
#     print("Sample Shape:", getattr(sample, 'shape', None))



# Training loop
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    # Testing loop
    model.eval()
    with torch.no_grad():
        total_loss = 0.0
        for inputs, targets in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()

        average_loss = total_loss / len(test_loader)
        print(f"Epoch {epoch+1}/{num_epochs}, Test Loss: {average_loss:.4f}")